In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2Config

In [2]:
config = GPT2Config()
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.28.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [3]:
class GPT2Attention(nn.Module):
    def __init__(self, config):
        super().__init__()

        max_positions = config.n_positions
        self.mask = torch.tril(torch.ones(max_positions, max_positions), dtype = torch.bool).view(1, 1, max_positions, max_positions)
        self.embed_dim = config.n_embd
        self.num_heads = config.n_head
        self.head_dim = self.embed_dim // self.num_heads
        self.split_size = self.embed_dim
        self.c_attn = nn.Linear(self.embed_dim, 3 * self.embed_dim)
        self.c_proj = nn.Linear(self.embed_dim, self.embed_dim)
        self.dropout = nn.Dropout(0.1)

    def _attn(self, query, key, value):
        # query, key, value: [batch_size, num_heads, seq_len, head_dim]
        attn_weights = torch.matmul(query, key.transpose(-1, -2))
        attn_weights = attn_weights / (float(self.head_dim) ** 0.5)

        # sequence length
        T = query.size(-2) # dimensionality before the last one
        casual_mask = self.mask[:, :, :T, :T].bool()
        attn_weights = torch.where(casual_mask, attn_weights, torch.tensor(-1e4))

        attn_weights = nn.Softmax(dim = -1)(attn_weights)
        attn_weights = self.dropout(attn_weights)
        # attn_weights: [batch_size, num_heads, seq_len, seq_len] matmul [batch_size, num_heads, seq_len, head_dim]
        attn_output = torch.matmul(attn_weights, value)
        return attn_output
        
    def forward(self, x):
        # x of shape [batch_size (B), seq_len (T), embed_dim (C)]
        B, T, C = x.size()
        query, key, value = self.c_attn(x).split(self.split_size, dim = -1) # (B,T,C) -> (B,T,3*C)
        query = query.view(B, T, self.num_heads, self.head_dim).transpose(1, 2) # (B,T,3*C) -> (B,T,num_heads,head_dim) -> (B,num_heads,T,head_dim)
        key = key.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        value = value.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # query, key, value: [batch_size, num_heads, seq_len, head_dim]
        attn_output = self._attn(query, key, value) # [batch_size, num_heads, seq_len, head_dim]
        attn_output = attn_output.transpose(1, 2).view(B, T, C) # [batch_size, seq_len, embed_dim]
        attn_output = self.c_proj(attn_output) # [batch_size, seq_len, embed_dim]
        attn_output = self.dropout(attn_output)
        return attn_output

In [4]:
class GPT2MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.n_embd
        self.mlp = nn.Sequential(nn.Linear(embed_dim, 4 * embed_dim), nn.GELU(), nn.Linear(4 * embed_dim, embed_dim), nn.Dropout(0.1))

    def forward(self, x):
        return self.mlp(x)

In [5]:
class GPT2Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.n_embd
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.attn = GPT2Attention(config)
        self.mlp = GPT2MLP(config)

    def forward(self, hidden_states):
        residual = hidden_states
        hidden_states = self.ln1(hidden_states)
        attn_outputs = self.attn(hidden_states)
        hidden_states = residual + attn_outputs
        residual = hidden_states
        feed_forward_hidden_states = self.mlp(hidden_states)
        hidden_states = residual + feed_forward_hidden_states
        return hidden_states


In [ ]:
class GPT2Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed_dim = config.n_embd
        self.vocab_size = config.vocab_size

        self.wte = nn.Embedding(self.vocab_size, self.embed_dim)
        self.wpe = nn.Embedding(config.n_positions, self.embed_dim)

        self.drop = nn.Dropout(0.1)
        self.blocks = nn.ModuleList([GPT2Block(config) for _ in range(config.n_layer)])

        self.ln_f = nn.LayerNorm(self.embed_dim)

    def forward(self, input_ids = None, position_ids = None):
        # input_ids: [batch_size, seq_len]
        input_shape = input_ids.size()
        batch_size = input_ids.size()
        device = input_ids.device

        position_ids = torch.arange(input_shape[-1], dtype = torch.long, device = device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)

        hidden_states = inputs_embeds + position_embeds
        hidden_states = self.drop(hidden_states)

        for block in self.blocks:
            hidden_states = block(hidden_states)

        hidden_states = self.ln_f(hidden_states)
        hidden_states = hidden_states.view(*input_shape, -1)
        return hidden_states